In [105]:
from pyhmy import signing, staking_signing, numbers, transaction, account, validator as validator_module, staking_structures, contract
from web3 import Web3

In [3]:
# we need five transactions in conftest
# simple transfer (from localnet address)
# contract creation (from second address)
# cross shard transfer (from second address)
# validator creation (from localnet address)
# delegation (from second address)

### Simple Transfer

In [144]:
pk = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
tx = {
    'from': 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3',
    # 3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65
    'to': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gas': 21000,
    'chainId': 2, # localnet
    'value': int( numbers.convert_one_to_atto( 503 ) ),
    'nonce': 0,
    'shardID': 0,
    'toShardID': 0,
}
raw_tx = signing.sign_transaction(tx, pk).rawTransaction.hex()
tx_hash = transaction.send_raw_transaction(raw_tx)

In [145]:
print( raw_tx )
print( tx_hash )

0xf86f8085174876e8008252088080941f2213a52f7409ff4f103458e6d202e0b3aa805a891b4486fafde57c00008027a0d7c0b20207dcc9dde376822dc3f5625eac6f59a7526111695cdba3e29553ca17a05d4ca9a421ae16f89cbf6848186eaea7a800da732446dff9952e7c1e91d414e3
0xc26be5776aa57438bccf196671a2d34f3f22c9c983c0f844c62b2fb90403aa43


### Contract Creation

In [147]:
pk = '3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65'
data = "0x6080604052348015600f57600080fd5b50607780601d6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f0033"
tx = {
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gas': 100000,
    'chainId': 2,
    'nonce': 0,
    'shardID': 0,
    'toShardID': 0,
    'data': data,
}
raw_tx = signing.sign_transaction(tx, pk).rawTransaction.hex()
tx_hash = transaction.send_raw_transaction(raw_tx)

In [148]:
print( raw_tx )
print( tx_hash )

0xf8e88085174876e800830186a080808080b8946080604052348015600f57600080fd5b50607780601d6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f003327a08bf26ee0120c296b17af507f62606abdb5c5f09a65642c3d30b349b8bfbb3d69a03ec7be51c615bcbf2f1d63f6eaa56cf8d7be81671717f90239619830a81ebc9f
0xa605852dd2fa39ed42e101c17aaca9d344d352ba9b24b14b9af94ec9cb58b31f


In [130]:
contract_address = transaction.get_transaction_receipt( tx_hash ).get( 'contractAddress' )
deployed = contract.get_code( contract_address, 'latest' )

In [131]:
print( deployed )
print( data )

0x6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f0033
0x6080604052348015600f57600080fd5b50607780601d6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f0033


In [132]:
print( "".join( [ deployed[ len( deployed ) - ( i + 1 ) ] for i in range( len( deployed ) ) ] ) )
print( "".join( [ data[ len( data ) - ( i + 1 ) ] for i in range( len( data ) ) ] ) )

3300f080003436c6f63746dc447adeb4d82da06660bc16489e70d315a93fde5dd4f95a1387620d8e0af3af0221228537660796462aef3f091002063009180915092518100615080406b5df080006b575d2064163dc63943608c10e065300067582060163400605b5df08000675f0065108432504060806x0
3300f080003436c6f63746dc447adeb4d82da06660bc16489e70d315a93fde5dd4f95a1387620d8e0af3af0221228537660796462aef3f091002063009180915092518100615080406b5df080006b575d2064163dc63943608c10e065300067582060163400605b5df08000675f0065108432504060806ef3f0006930006d10608770605b5df08000675f0065108432504060806x0


In [133]:
"0x6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f0033" == deployed

True

### Cross Shard Transfer

In [149]:
pk = '3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65'
tx = {
    'from': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gas': 21000,
    'chainId': 2,
    'nonce': 1,
    'shardID': 0,
    'toShardID': 1,
    'to': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
    'value': Web3.toWei( 100, 'gwei' ),
}
raw_tx = signing.sign_transaction(tx, pk).rawTransaction.hex()
tx_hash = transaction.send_raw_transaction(raw_tx)

In [150]:
print( raw_tx )
print( tx_hash )

0xf86b0185174876e800825208800194c9c6d47ee5f2e3e08d7367ad1a1373ba9dd1724185174876e8008027a02501c517220e9499f14e97c20b0a88cd3b7ba80637bba43ed295422e69a3f300a079b8e1213c9506184aed6ac2eb0b2cb00594c3f9fcdd6c088937ce17fe47107c
0xf73ba634cb96fc0e3e2c9d3b4c91379e223741be4a5aa56e6d6caf49c1ae75cf


In [153]:
account.get_balance( 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes', 'http://localhost:9502' )

0

### Validator Creation

In [154]:
pk = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
address = "one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3"
info = {
    "name": "Alice",
    "identity": "alice",
    "website": "alice.harmony.one",
    "security-contact": "Bob",
    "details": "Are you even reading this?",
    "min-self-delegation": int( numbers.convert_one_to_atto( 10000 ) ),
    "max-total-delegation": int( numbers.convert_one_to_atto( 100000 ) ),
    "rate": "0.1",
    "max-rate": "0.9",
    "max-change-rate": "0.05",
    "bls-public-keys": [
        # private key is b1f2a5029f5f43c8c933a61ce936ced030b2c9379f8e2478fc888fa670cdbc89b8cd1ebc29b5b00a81d3152bb3aaa3a337404f50bee5e434430ca3693a94a1c102a765cf3b0887b8b0bcf5317d33f4bec60a97feae2498a39ab7a1c2
        # blspass.txt is empty
        "0xa20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712",
    ],
    "amount": int( numbers.convert_one_to_atto( 10000 ) ),
    "bls-key-sigs": [
        "0xef2c49a2f31fbbd23c21bc176eaf05cd0bebe6832033075d81fea7cff6f9bc1ab42f3b6895c5493fe645d8379d2eaa1413de55a9d3ce412a4f747cb57d52cc4da4754bfb2583ec9a41fe5dd48287f964f276336699959a5fcef3391dc24df00d",
    ]
}
validator = validator_module.Validator( address )
validator.load( info )
raw_tx = validator.sign_create_validator_transaction(
    1,
    Web3.toWei( 100, 'gwei' ),
    55000000,   # gas limit
    pk,
    2           # chain id
).rawTransaction.hex()
tx_hash = transaction.send_raw_staking_transaction(
        raw_tx,
)

In [155]:
print( raw_tx )
print( tx_hash )

0xf9017c80f9012994a5241513da9f4463f1d4874b548dfbac29d91f34f83d85416c69636585616c69636591616c6963652e6861726d6f6e792e6f6e6583426f629a41726520796f75206576656e2072656164696e6720746869733fddc988016345785d8a0000c9880c7d713b49da0000c887b1a2bc2ec500008a021e19e0c9bab24000008a152d02c7e14af6800000f1b0a20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712f862b860ef2c49a2f31fbbd23c21bc176eaf05cd0bebe6832033075d81fea7cff6f9bc1ab42f3b6895c5493fe645d8379d2eaa1413de55a9d3ce412a4f747cb57d52cc4da4754bfb2583ec9a41fe5dd48287f964f276336699959a5fcef3391dc24df00d8a021e19e0c9bab24000000185174876e8008403473bc028a08c1146305eaef981aa24c2f17c8519664d10c99ee42acedbc258749930d31a7ca031dadf114ee6ab9bd09933208094c65037b66c796bcfc57a70158106b37357b0
0x400e9831d358f5daccd153cad5bf53650a0d413bd8682ec0ffad55367d162968


### Delegation

In [156]:
pk = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"
tx = {
    'directive': staking_structures.Directive.Delegate,
    'delegatorAddress': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'validatorAddress': 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3',
    'amount': Web3.toWei( 100, 'ether' ),
    'nonce': 2,
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gasLimit': 50000,
    'chainId': 2,
}
raw_tx = staking_signing.sign_staking_transaction(
    tx,
    pk,
).rawTransaction.hex()
tx_hash = transaction.send_raw_staking_transaction(
        raw_tx,
)
print( raw_tx )
print( tx_hash )

0xf88302f4941f2213a52f7409ff4f103458e6d202e0b3aa805a94a5241513da9f4463f1d4874b548dfbac29d91f3489056bc75e2d631000000285174876e80082c35028a02c5e953062dcdfa2de9723639b63bab45705eb6dfbfe7f44536ed266c3c7ca20a0742964e646338e7431874f70715565d99c01c762324355c69db34a9ed9de81d7
0xc8177ace2049d9f4eb4a45fd6bd6b16f693573d036322c36774cc00d05a3e24f


### test_transaction.py - transfer 105 ONE to another address

In [157]:
pk = '3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65'
tx = {
    'from': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gas': 21000,
    'chainId': 2,
    'nonce': 3,
    'shardID': 0,
    'toShardID': 0,
    'to': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
    'value': Web3.toWei( 105, 'ether' ),
}
raw_tx = signing.sign_transaction(tx, pk).rawTransaction.hex()
tx_hash = transaction.send_raw_transaction(raw_tx)

In [158]:
print( raw_tx )
print( tx_hash )

0xf86f0385174876e800825208808094c9c6d47ee5f2e3e08d7367ad1a1373ba9dd172418905b12aefafa80400008027a07a4952b90bf38723a9197179a8e6d2e9b3a86fd6da4e66a9cf09fdc59783f757a053910798b311245525bd77d6119332458c2855102e4fb9e564f6a3b710d18bb0
0x7ccd80f8513f76ec58b357c7a82a12a95e025d88f1444e953f90e3d86e222571


### test_transaction.py - staking transaction

In [168]:
pk = "ff9ef6b00a61672b4b7bedd5ac653439b56ac8ee808c99a1bd871cf51b7d60eb"
tx = {
    'directive': staking_structures.Directive.Delegate,
    'delegatorAddress': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
    'validatorAddress': 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3',
    'amount': Web3.toWei( 100, 'ether' ),
    'nonce': 0,
    'gasPrice': Web3.toWei( 100, 'gwei' ),
    'gasLimit': 50000,
    'chainId': 2,
}
raw_tx = staking_signing.sign_staking_transaction(
    tx,
    pk,
).rawTransaction.hex()
tx_hash = transaction.send_raw_staking_transaction(
        raw_tx,
)
print( raw_tx )
print( tx_hash )

0xf88302f494c9c6d47ee5f2e3e08d7367ad1a1373ba9dd1724194a5241513da9f4463f1d4874b548dfbac29d91f3489056bc75e2d631000008085174876e80082c35027a0808ea7d27adf3b1f561e8da4676814084bb75ac541b616bece87c6446e6cc54ea02f19f0b14240354bd42ad60b0c7189873c0be87044e13072b0981a837ca76f64
0xe7d07ef6d9fca595a14ceb0ca917bece7bedb15efe662300e9334a32ac1da629


In [162]:
transaction.get_cx_receipt_by_hash( '0xf73ba634cb96fc0e3e2c9d3b4c91379e223741be4a5aa56e6d6caf49c1ae75cf', 'http://localhost:9502' )

{'blockHash': '0xf55f1fb3c9be76fb74370e8a7d8580327797d2d6082040074783207a171e2de6',
 'blockNumber': 34,
 'from': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
 'hash': '0xf73ba634cb96fc0e3e2c9d3b4c91379e223741be4a5aa56e6d6caf49c1ae75cf',
 'shardID': 0,
 'to': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
 'toShardID': 1,
 'value': 100000000000}

In [166]:
transaction.get_staking_transaction_by_hash( "0x279935278d20d778cbe4fdfa5d51be9eb1eb184053dc9a7cb88ad3365df73060" )

{'blockHash': '0x0000000000000000000000000000000000000000000000000000000000000000',
 'blockNumber': None,
 'from': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
 'gas': 50000,
 'gasPrice': 100000000000,
 'hash': '0x279935278d20d778cbe4fdfa5d51be9eb1eb184053dc9a7cb88ad3365df73060',
 'msg': {'amount': 100000000000000000000,
  'delegatorAddress': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
  'validatorAddress': 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3'},
 'nonce': 2,
 'r': '0x8660a63c10af06f2fb3f24b92cf61d4f319044a1f1931c4f4e54ce986ff563c',
 's': '0x597785559c4283d3ece2df37cbf37077966487a2a2dc0f4cdbbf75a8f20bc1a8',
 'timestamp': 0,
 'transactionIndex': 0,
 'type': 'Delegate',
 'v': '0x27'}